# Using Opik with Instructor

[Instructor](https://github.com/instructor-ai/instructor) is a Python library for working with structured outputs for LLMs built on top of Pydantic. It provides a simple way to manage schema validations, retries and streaming responses.

## Creating an account on Comet.com

[Comet](https://www.comet.com/site?from=llm&utm_source=opik&utm_medium=colab&utm_content=haystack&utm_campaign=opik) provides a hosted version of the Opik platform, [simply create an account](https://www.comet.com/signup?from=llm&utm_source=opik&utm_medium=colab&utm_content=haystack&utm_campaign=opik) and grab you API Key.

> You can also run the Opik platform locally, see the [installation guide](https://www.comet.com/docs/opik/self-host/overview/?from=llm&utm_source=opik&utm_medium=colab&utm_content=haystack&utm_campaign=opik) for more information.

In [ ]:
%pip install --upgrade --quiet opik instructor anthropic

In [1]:
import opik

opik.configure(use_local=False)

OPIK: Opik is already configured. You can check the settings by viewing the config file at /Users/jacquesverre/.opik.config


For this demo we are going to use an OpenAI so we will need to configure our OpenAI API key:

In [2]:
import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

## Using Opik with Instructor library

In order to log traces from Instructor into Opik, we are going to patch the `instructor` library. This will log each LLM call to the Opik platform.

For all the integrations, we will first add tracking to the LLM client and then pass it to the Instructor library:

In [5]:
from opik.integrations.openai import track_openai
import instructor
from pydantic import BaseModel
from openai import OpenAI


# We will first create the OpenAI client and add the `track_openai`
# method to log data to Opik
openai_client = track_openai(OpenAI())

# Patch the OpenAI client for Instructor
client = instructor.from_openai(openai_client)


# Define your desired output structure
class UserInfo(BaseModel):
    name: str
    age: int


user_info = client.chat.completions.create(
    model="gpt-4o-mini",
    response_model=UserInfo,
    messages=[{"role": "user", "content": "John Doe is 30 years old."}],
)

print(user_info)

name='John Doe' age=30


Thanks to the `track_openai` method, all the calls made to OpenAI will be logged to the Opik platform. This approach also works well if you are also using the `opik.track` decorator as it will automatically log the LLM call made with Instructor to the relevant trace.

![Trace screenshot](https://raw.githubusercontent.com/comet-ml/opik/main/apps/opik-documentation/documentation/fern/img/cookbook/instructor_cookbook.png)

## Integrating with other LLM providers

The instructor library supports many LLM providers including OpenAI, Bedrock, Gemini, etc. Opik supports the majority of these.

Here are some code snippets needed for the integration.

### OpenAI

In [6]:
from opik.integrations.openai import track_openai
import instructor
from openai import OpenAI

# Add Opik tracking
openai_client = track_openai(OpenAI())

# Patch the OpenAI client for Instructor
client = instructor.from_openai(openai_client)

### Anthropic

In [ ]:
from opik.integrations.anthropic import track_anthropic
import instructor
from anthropic import Anthropic

# Add Opik tracking
anthropic_client = track_anthropic(Anthropic())

# Patch the Anthropic client for Instructor
client = instructor.from_openai(anthropic_client)

### Gemini

In [ ]:
from opik.integrations.genai import track_genai
import instructor
import google.generativeai as genai

# Add Opik tracking
genai_client = track_genai(
    genai.GenerativeModel(
        model_name="models/gemini-1.5-flash-latest",
    )
)

# Patch the GenAI client for Instructor
client = instructor.from_openai(genai_client, mode=instructor.Mode.GEMINI_JSON)